In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd

In [2]:
# def grb_prob(number_grb_year, delta_t = 1):
#     number_grb_day = number_grb_year/365.25
#     print('GRB:', number_grb_day*delta_t)
#     return number_grb_day * delta_t * np.exp(-number_grb_day*delta_t) 

# def sne_prob(number_sne_year, delta_t = 1, fraction_rate = 0.21):
#     number_sne_day = number_sne_year/365.25 * fraction_rate
#     # print(number_sne_day*delta_t)
#     return number_sne_day * delta_t * np.exp(-number_sne_day*delta_t)

def grb_prob(number_grb_year, delta_t = 1):
    number_grb_day = number_grb_year/365.25
    # print('GRB:', number_grb_day*delta_t)
    return 1 - np.exp(-number_grb_day*delta_t), number_grb_day*delta_t

def sne_prob(number_sne_year, delta_t = 1, fraction_rate = 0.21):
    number_sne_day = number_sne_year/365.25 * fraction_rate
    # print(number_sne_day)
    return 1 - np.exp(-number_sne_day*delta_t), number_sne_day*delta_t

# def probby(number_grb_year, number_sne_year, delta_t = 1, fraction_rate = 0.21):
#     number_grb_day = number_grb_year/365.25
#     number_sne_day = number_sne_year/365 * fraction_rate
    
#     part1 = (1-np.exp(-delta_t*number_sne_day)) * number_grb_day
#     part2 = (1-np.exp(-delta_t*number_grb_day)) * number_sne_day
    
#     print(part1, part2)
    
#     return ( 1/(number_sne_day + number_grb_day) ) * (part1 + part2)

In [3]:
file = '/Users/zgl12/Downloads/Summary_table (1).txt'

df  = pd.read_csv(file, delim_whitespace=True)

df

lengths = []

for i in range(11, 25):

    temp_df = df #df[df['GRB_name_Fermi'] != 'None']

    lengths.append(len(temp_df[temp_df['GRB_name'].str.startswith(f'GRB{i}')]))
    
grbs_year = np.nanmean(lengths)
grbs_year_err = np.nanstd(lengths, ddof = 1)

/var/folders/6v/pg_c9mqn3gn45gn_zmvy9w9cvfsx9j/T/ipykernel_17186/1138129278.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df  = pd.read_csv(file, delim_whitespace=True)


In [4]:
np.nansum(lengths)

4264

In [5]:
# dt = 0.6

# gProb = grb_prob(grbs_year, delta_t = dt)
# sProb = sne_prob(6368/3, delta_t = dt, fraction_rate = 0.1182)

# temp_Prob = gProb * sProb * dt

# temp_Prob

grbs_year, grbs_year_err

(304.57142857142856, 13.042618222960071)

In [17]:
dt = 0.3
sne_val = 6368/3 * 0.21
# sne_val = 487.62516272856794

sProb, mu_sne = sne_prob(sne_val, delta_t = dt, fraction_rate = 1)

# for frac in tqdm(np.linspace(5, 95, 91)):
#     gProb, mu_grb = grb_prob(grbs_year/2, delta_t = dt)

from tqdm import tqdm
probs = []
o_probs = []
    
xrb_fracs = np.linspace(0, 1, 101)
for frac in tqdm(xrb_fracs, desc="Calculating probabilities"):
    prob, _ = grb_prob(frac*grbs_year, delta_t = dt)
    probs.append(prob)
    
    o_probs.append((prob*sne_val + sProb*frac*grbs_year) / (sne_val + frac*grbs_year))
    
    
    
import scipy.stats as stats

probby = np.trapz(probs, xrb_fracs)
confidence_level = 1- probby # * (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for a {probby*100:.3f}% confidence level is: {z_score:.3f}")
print(sProb)

probby = np.trapz(o_probs, xrb_fracs)
confidence_level = 1- probby # * (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for a {probby*100:.3f}% confidence level is: {z_score:.3f}")


prob, _ = grb_prob(0.5*grbs_year, delta_t = dt)

print((prob*sne_val + 0.5*grbs_year) / (sne_val + 0.5*grbs_year))

# print(sProb, gProb)

# temp_Prob = gProb * sProb * 2

# temp_Prob

# 1 - (sProb*mu_grb + gProb*mu_sne)

Calculating probabilities: 100%|██████████| 101/101 [00:00<00:00, 376890.31it/s]

The z-score for a 11.527% confidence level is: 1.575
0.3065854830547303
The z-score for a 15.347% confidence level is: 1.427
0.34227421494104954


In [ ]:
# Define the confidence level
confidence_level_50 = 1- 0.2989104911098284 * (1 - 0.99394678)
confidence_level_90 = 1- 0.4304413201526259 * (1 - 0.99394678)
confidence_level_95 = 1- 0.4421225710543031 * (1 - 0.99394678)
confidence_level_99 = 1- 0.4515929414218028 * (1 - 0.99394678)
confidence_level_10 = 1- 0.0784731323304526 * (1 - 0.99394678)
confidence_level_05 = 1- 0.0406033155108535 * (1 - 0.99394678)
confidence_level_01 = 1- 0.008401252894112901 * (1 - 0.99394678)

# # Convert to z-score (sigma level) using the PPF function
z_score_50 = stats.norm.ppf((1 + confidence_level_50) / 2)
z_score_90 = stats.norm.ppf((1 + confidence_level_90) / 2)
z_score_95 = stats.norm.ppf((1 + confidence_level_95) / 2)
z_score_99 = stats.norm.ppf((1 + confidence_level_99) / 2)
z_score_10 = stats.norm.ppf((1 + confidence_level_10) / 2)
z_score_05 = stats.norm.ppf((1 + confidence_level_05) / 2)
z_score_01 = stats.norm.ppf((1 + confidence_level_01) / 2)

# # Output the result
print(f"The z-score for a {confidence_level_50*100:.3f}% confidence level is: {z_score_50:.3f}, {50}%")
print(f"The z-score for a {confidence_level_90*100:.3f}% confidence level is: {z_score_90:.3f}, {90}%")
print(f"The z-score for a {confidence_level_95*100:.3f}% confidence level is: {z_score_95:.3f}, {95}%")
print(f"The z-score for a {confidence_level_99*100:.3f}% confidence level is: {z_score_99:.3f}, {99}%")
print(f"The z-score for a {confidence_level_10*100:.3f}% confidence level is: {z_score_10:.3f}, {10}%")
print(f"The z-score for a {confidence_level_05*100:.3f}% confidence level is: {z_score_05:.3f}, {5}%")
print(f"The z-score for a {confidence_level_01*100:.3f}% confidence level is: {z_score_01:.3f}, {1}%")

In [ ]:
confidence_level = 1- 0.15321393130481006 * (1 - 0.98557172)

z_score = stats.norm.ppf((1 + confidence_level) / 2)

# # Output the result
print(f"The z-score for a {confidence_level*100:.3f}% confidence level is: {z_score:.3f}")

In [ ]:
from astropy.cosmology import FlatLambdaCDM
import numpy as np

cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

dL = cosmo.luminosity_distance(0.019).value * 3.086e24

time = 7.424
c = 2.99792458e10
v = 2.99792458e10
radius = time * c

v0 = 1e9

print(radius)

E_inf = radius**2 * 8 * np.pi * v0 * c / 0.34

print(E_inf)



In [ ]:
np.sqrt(4*np.pi*dL**2 * 5.0855e-7 / (c / 0.34) / (8*np.pi ) / 1e9)

In [ ]:
np.sqrt((0.005*dL**2 * 5.0855e-7)*0.34 / (8*np.pi*1e9) / c**3)

In [ ]:
456.39251406341475/60